https://developers.kakao.com/docs/latest/ko/local/dev-guide#address-coord

In [79]:
import pandas as pd
import requests
import json
# url ='https://dapi.kakao.com/v2/local/search/category.${FORMAT}'
# url ='https://dapi.kakao.com/v2/local/search/category.json?category_group_code=PM9&radius=2000'
# 36.993883,127.089649

# searching ='.json?x=36.993883&y=127.089649&radius=2000&category_group_code=PM9'
# url ='https://dapi.kakao.com/v2/local/search/category.${}'.format(searching)

# url ='https://dapi.kakao.com/v2/local/search/category.json?x=36.916583&y=127.132933&radius=2000&category_group_code=PM9'

# url = 'https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=127.0003429871931&y=37.57113559406886'



# headers = {'Authorization': 'KakaoAK {}'.format('c454c0e64688ce2bde2dfff9cceced87')}
# data = requests.post(url,headers=headers)
# result = json.loads(data.text)
# result


# documents = result['documents']
# documents
# for data in documents:
  # 127.0003429871931 ,37.57113559406886

### 
# 지번 -> 위도 경도
# searching ='입장면 독정리 290 0'
searching =  '천안시 서북구	입장면 340 15'
url = 'https://dapi.kakao.com/v2/local/search/address.json?query={}'.format(searching)


headers = {'Authorization': 'KakaoAK {}'.format('c454c0e64688ce2bde2dfff9cceced87') }
data = requests.post(url,headers=headers)
result = json.loads(data.text)
result
documents = result['documents']
documents[0]
# print(documents)
# for data in documents:
#   print(data['address'])  
  # 127.0003429871931 ,37.57113559406886


KeyError: 'documents'

In [1]:
import pandas as pd
import requests
import json


c:\Users\Lee\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [2]:
addressData = pd.read_csv(r'C:\Users\Lee\Desktop\Project\천안시 시각화\천안시_DataDir\202307_도로명주소 한글_전체분\충청남도지번.txt',encoding='cp949',sep='|',header=None)
col_list= ['관리번호','일련번호','법정동코드','시도명','시군구명','법정읍면동명','법정리명','산여부','번지','호','대표여부']
addressData.columns=col_list
addressData = addressData[addressData['시군구명'].isin(['천안시 동남구','천안시 서북구'])]
addressData

,관리번호,일련번호,법정동코드,시도명,시군구명,법정읍면동명,법정리명,산여부,번지,호,대표여부
0,4413025629101030004000001,1,4413325629,충청남도,천안시 서북구,직산읍,모시리,0,103,4,1
1,4413025632102760000000001,1,4413325632,충청남도,천안시 서북구,직산읍,신갈리,0,276,0,1
2,4413034027103000000000001,1,4413134027,충청남도,천안시 동남구,성남면,신사리,0,300,0,1
3,4413034027103000000000001,3,4413134027,충청남도,천안시 동남구,성남면,신사리,0,300,3,0
4,4413034027103000000000001,5,4413134027,충청남도,천안시 동남구,성남면,신사리,0,300,5,0
...,...,...,...,...,...,...,...,...,...,...,...
93506,4413331038102740103012993,5,4413331038,충청남도,천안시 서북구,입장면,독정리,0,274,3,0
93507,4413331038102740103012993,6,4413331038,충청남도,천안시 서북구,입장면,독정리,0,274,4,0
93508,4413331038102840000013127,1,4413331038,충청남도,천안시 서북구,입장면,독정리,0,284,0,1
93509,4413331038102870000015693,1,4413331038,충청남도,천안시 서북구,입장면,독정리,0,287,0,1


In [ ]:
# 법정리명      nan값 주의
from tqdm import tqdm

cnt=0

addressCopy = addressData.copy()
# addressCopy = addressCopy[:10]
headers = {'Authorization': 'KakaoAK {}'.format('c454c0e64688ce2bde2dfff9cceced87') }
addressCopy['byAPI'] =None
for idx in tqdm(range(len(addressCopy))):
  try:
    address1 = addressCopy.loc[idx,'시군구명']
    address2 = addressCopy.loc[idx,'법정읍면동명']
    address3 = addressCopy.loc[idx,'번지']
    address4 = addressCopy.loc[idx,'호']
    address =  address1 + ' ' + address2 + ' ' + str(address3) + ' ' + str(address4)

    searching = address
    url = 'https://dapi.kakao.com/v2/local/search/address.json?query={}'.format(searching)
    # headers = {'Authorization': 'KakaoAK {}'.format('c454c0e64688ce2bde2dfff9cceced87') }
    data = requests.post(url,headers=headers)
    result = json.loads(data.text)
    documents = result['documents']
    

    addressCopy.at[idx,'byAPI'] = documents[0]
  except:
    cnt+=1
    print('쿼리값을 못찾는중:',cnt)
    pass

In [53]:
# C:\Users\Lee\Desktop\Project\천안시 시각화\천안시_DataDir
addressCopy.to_csv(r'C:\Users\Lee\Desktop\Project\천안시 시각화\천안시_DataDir\addressByAPI.csv',index=False)

In [62]:
addressCopy.to_pickle(r'C:\Users\Lee\Desktop\Project\천안시 시각화\천안시_DataDir\addressByAPIpickle.pickle')

In [68]:
t2 = pd.read_pickle(r'C:\Users\Lee\Desktop\Project\천안시 시각화\천안시_DataDir\addressByAPIpickle.pickle')
t2[['byAPI']].iloc[0][0]['address']

{'address_name': '충남 천안시 서북구 직산읍 모시리 103-4',
 'b_code': '4413325629',
 'h_code': '4413325600',
 'main_address_no': '103',
 'mountain_yn': 'N',
 'region_1depth_name': '충남',
 'region_2depth_name': '천안시 서북구',
 'region_3depth_h_name': '직산읍',
 'region_3depth_name': '직산읍 모시리',
 'sub_address_no': '4',
 'x': '127.136346304957',
 'y': '36.8707318787372'}

In [54]:
t = pd.read_csv(r'C:\Users\Lee\Desktop\Project\천안시 시각화\천안시_DataDir\addressByAPI.csv')
t

<ipython-input-54-aaf86605ee67>:1: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  t = pd.read_csv(r'C:\Users\Lee\Desktop\Project\천안시 시각화\천안시_DataDir\addressByAPI.csv')


,관리번호,일련번호,법정동코드,시도명,시군구명,법정읍면동명,법정리명,산여부,번지,호,대표여부,byAPI
0,4413025629101030004000001,1,4413325629,충청남도,천안시 서북구,직산읍,모시리,0,103,4,1,{'address': {'address_name': '충남 천안시 서북구 직산읍 모...
1,4413025632102760000000001,1,4413325632,충청남도,천안시 서북구,직산읍,신갈리,0,276,0,1,{'address': {'address_name': '충남 천안시 서북구 직산읍 군...
2,4413034027103000000000001,1,4413134027,충청남도,천안시 동남구,성남면,신사리,0,300,0,1,{'address': {'address_name': '충남 천안시 동남구 성남면 대...
3,4413034027103000000000001,3,4413134027,충청남도,천안시 동남구,성남면,신사리,0,300,3,0,{'address': {'address_name': '충남 천안시 동남구 성남면 신...
4,4413034027103000000000001,5,4413134027,충청남도,천안시 동남구,성남면,신사리,0,300,5,0,{'address': {'address_name': '충남 천안시 동남구 성남면 신...
...,...,...,...,...,...,...,...,...,...,...,...,...
93506,4413331038102740103012993,5,4413331038,충청남도,천안시 서북구,입장면,독정리,0,274,3,0,NaN
93507,4413331038102740103012993,6,4413331038,충청남도,천안시 서북구,입장면,독정리,0,274,4,0,NaN
93508,4413331038102840000013127,1,4413331038,충청남도,천안시 서북구,입장면,독정리,0,284,0,1,NaN
93509,4413331038102870000015693,1,4413331038,충청남도,천안시 서북구,입장면,독정리,0,287,0,1,NaN


In [134]:
t[t['법정읍면동명']=='입장면'].sample(10)


,관리번호,일련번호,법정동코드,시도명,시군구명,법정읍면동명,법정리명,산여부,번지,호,대표여부,byAPI
93191,4413331038100200007014742,1,4413331038,충청남도,천안시 서북구,입장면,독정리,0,20,7,1,NaN
91573,4413331031102840000018983,1,4413331031,충청남도,천안시 서북구,입장면,신두리,0,284,0,1,NaN
90300,4413331028100170001021693,3,4413331028,충청남도,천안시 서북구,입장면,시장리,0,300,0,0,NaN
92521,4413331036101510001008709,1,4413331036,충청남도,천안시 서북구,입장면,가산리,0,151,1,1,NaN
88449,4413331021101690000013920,1,4413331021,충청남도,천안시 서북구,입장면,하장리,0,167,0,1,NaN
89086,4413331024100470012014792,1,4413331024,충청남도,천안시 서북구,입장면,도림리,0,47,12,1,NaN
90523,4413331028101980005026146,6,4413331028,충청남도,천안시 서북구,입장면,시장리,0,198,12,0,NaN
91626,4413331031103930022014971,3,4413331031,충청남도,천안시 서북구,입장면,신두리,0,439,8,0,NaN
87975,4413331021100230006013063,1,4413331021,충청남도,천안시 서북구,입장면,하장리,0,23,6,1,NaN
91608,4413331031103820001013325,2,4413331031,충청남도,천안시 서북구,입장면,신두리,0,431,2,0,NaN


In [ ]:
천안시 서북구	입장면 340 15

In [69]:
t[['byAPI']].iloc[0][0]['address']

TypeError: string indices must be integers

In [52]:
addressCopy.loc[:10,'byAPI'][4]
# 두정서해그랑블아파트	


{'address': {'address_name': '충남 천안시 서북구 직산읍 군동리 276',
  'b_code': '4413325621',
  'h_code': '4413325600',
  'main_address_no': '276',
  'mountain_yn': 'N',
  'region_1depth_name': '충남',
  'region_2depth_name': '천안시 서북구',
  'region_3depth_h_name': '직산읍',
  'region_3depth_name': '직산읍 군동리',
  'sub_address_no': '',
  'x': '127.166707737053',
  'y': '36.9069639060011'},
 'address_name': '충남 천안시 서북구 직산읍 군동리 276',
 'address_type': 'REGION_ADDR',
 'road_address': {'address_name': '충남 천안시 서북구 직산읍 성진로 309-15',
  'building_name': '(주)두호스테인레스',
  'main_building_no': '309',
  'region_1depth_name': '충남',
  'region_2depth_name': '천안시 서북구',
  'region_3depth_name': '직산읍 군동리',
  'road_name': '성진로',
  'sub_building_no': '15',
  'underground_yn': 'N',
  'x': '127.166496744125',
  'y': '36.9069963709786',
  'zone_no': '31032'},
 'x': '127.166707737053',
 'y': '36.9069639060011'}

# 좌표 -> category

In [26]:
x = addressCopy['byAPI'][0]['address']['x']
y = addressCopy['byAPI'][0]['address']['y']

searching ='category_group_code={code}&x={x}&y={y}&radius={radius}'.format(code='CS2',x=str(x),y=str(y),radius=2000)
# searching ='{code}&{radius}'.format(code='PM9',radius=2)

url='https://dapi.kakao.com/v2/local/search/category.json?{}'.format(searching)

# url='https://dapi.kakao.com/v2/local/search/category.json?category_group_code=PM9&radius=2000'


headers = {'Authorization': 'KakaoAK {}'.format('c454c0e64688ce2bde2dfff9cceced87') }

data=requests.post(url,headers=headers)
result = json.loads(data.text)
result
# documents = result['documents']
# documents

{'documents': [{'address_name': '충남 천안시 서북구 직산읍 부송리 40-1',
   'category_group_code': 'CS2',
   'category_group_name': '편의점',
   'category_name': '가정,생활 > 편의점 > 미니스톱',
   'distance': '459',
   'id': '1798386187',
   'phone': '1577-9621',
   'place_name': '미니스톱 천안부송산단점',
   'place_url': 'http://place.map.kakao.com/1798386187',
   'road_address_name': '충남 천안시 서북구 직산읍 부송1길 7',
   'x': '127.1382046146598',
   'y': '36.874597978857075'},
  {'address_name': '충남 천안시 서북구 직산읍 부송리 136',
   'category_group_code': 'CS2',
   'category_group_name': '편의점',
   'category_name': '가정,생활 > 편의점 > CU',
   'distance': '706',
   'id': '36638305',
   'phone': '1577-8007',
   'place_name': 'CU 천안직산공단점',
   'place_url': 'http://place.map.kakao.com/36638305',
   'road_address_name': '충남 천안시 서북구 직산읍 부송4길 5',
   'x': '127.131707131928',
   'y': '36.875900238685'},
  {'address_name': '충남 천안시 서북구 직산읍 모시리 322',
   'category_group_code': 'CS2',
   'category_group_name': '편의점',
   'category_name': '가정,생활 > 편의점 > GS25',
 

In [43]:
df = pd.read_csv(r'C:\Users\Lee\Desktop\Project\천안시 시각화\천안시_DataDir\202307_도로명주소 한글_전체분\202307_도로명주소 한글_전체분.txt',encoding='cp949',sep='|',header=None)
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,44131120100012500001700003,4413112000,충청남도,천안시 동남구,구룡동,NaN,0,418,11,441311000125,0,17,3,NaN
1,44131120100012500001700003,4413112000,충청남도,천안시 동남구,구룡동,NaN,0,418,14,441311000125,0,17,3,NaN
2,44131120100012500001700003,4413112000,충청남도,천안시 동남구,구룡동,NaN,0,418,13,441311000125,0,17,3,NaN
3,44131120100012500001800000,4413112000,충청남도,천안시 동남구,구룡동,NaN,0,652,6,441311000125,0,18,0,NaN
4,44131120100012500001800000,4413112000,충청남도,천안시 동남구,구룡동,NaN,0,651,3,441311000125,0,18,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
185919,44825250486137800001900004,4482525021,충청남도,태안군,태안읍,동문리,0,746,2,448254861378,0,19,4,NaN
185920,44825250486137800001900005,4482525021,충청남도,태안군,태안읍,동문리,0,841,0,448254861378,0,19,5,NaN
185921,44825250486137800001900005,4482525021,충청남도,태안군,태안읍,동문리,1,16,6,448254861378,0,19,5,NaN
185922,44825250486137800001900005,4482525021,충청남도,태안군,태안읍,동문리,0,875,0,448254861378,0,19,5,NaN


In [50]:
col_list= ['관리번호','일련번호','법정동코드','시도명','시군구명','법정읍면동명','법정리명','산여부','번지','호','대표여부']
df.columns=col_list
df

,관리번호,일련번호,법정동코드,시도명,시군구명,법정읍면동명,법정리명,산여부,번지,호,대표여부
0,4413025629101030004000001,1,4413325629,충청남도,천안시 서북구,직산읍,모시리,0,103,4,1
1,4413025632102760000000001,1,4413325632,충청남도,천안시 서북구,직산읍,신갈리,0,276,0,1
2,4413034027103000000000001,1,4413134027,충청남도,천안시 동남구,성남면,신사리,0,300,0,1
3,4413034027103000000000001,3,4413134027,충청남도,천안시 동남구,성남면,신사리,0,300,3,0
4,4413034027103000000000001,5,4413134027,충청남도,천안시 동남구,성남면,신사리,0,300,5,0
...,...,...,...,...,...,...,...,...,...,...,...
675683,4483040033105360000077007,1,4427039033,충청남도,당진시,송산면,가곡리,0,536,0,1
675684,4483040033105360000077007,2,4427039033,충청남도,당진시,송산면,가곡리,0,537,0,0
675685,4483040033200110001075209,1,4427039033,충청남도,당진시,송산면,가곡리,0,373,37,1
675686,4483040033200110001075209,2,4427039033,충청남도,당진시,송산면,가곡리,0,373,39,0


In [72]:
df['시군구명'].value_counts()
addressDB = df[df['시군구명'].isin(['천안시 동남구','천안시 서북구'])]
# 입장면 독정리 290 0

In [57]:
# df[df['관리번호']=='4413025632102760000000001']
df.isna().sum()

관리번호           0
일련번호           0
법정동코드          0
시도명            0
시군구명           0
법정읍면동명         0
법정리명      104196
산여부            0
번지             0
호              0
대표여부           0
dtype: int64

In [58]:
부가정보 = pd.read_csv(r'C:\Users\Lee\Desktop\Project\천안시 시각화\천안시_DataDir\202307_도로명주소 한글_전체분\충청남도부가정보.txt',encoding='cp949',sep='|',header=None)
부가정보

,0,1,2,3,4,5,6,7,8
0,4413025629101030004000001,4.413326e+09,직산읍,31039,NaN,NaN,NaN,NaN,0
1,4413025632102760000000001,4.413326e+09,직산읍,31040,NaN,NaN,NaN,(주)대성이앤지,0
2,4413034027103000000000001,4.413134e+09,성남면,31245,NaN,NaN,NaN,NaN,0
3,4413110100100010001010693,4.413151e+09,중앙동,31128,NaN,NaN,NaN,NaN,0
4,4413110100100010003014920,4.413151e+09,중앙동,31128,NaN,NaN,NaN,두나무빌딩,0
...,...,...,...,...,...,...,...,...,...
489753,4483040033103940031075072,4.427039e+09,송산면,31709,NaN,NaN,NaN,NaN,0
489754,4483040033104990001075556,4.427039e+09,송산면,31709,NaN,NaN,NaN,당진시자원순환센터,0
489755,4483040033105320000077072,4.427039e+09,송산면,31709,NaN,NaN,NaN,관리실,0
489756,4483040033105360000077007,4.427039e+09,송산면,31709,NaN,NaN,NaN,NaN,0


In [62]:
부가정보.loc[:,7].isna().sum()

449543